# Automatic Accountant Assistant User Manual

Zeng, Nanjin  
master student of 20/21 | Quantitative Economics | National University of Singapore

## 1. Introduction

Automatic accountant assistant is an accounting entries collection and analysis system for commercial companies. Operator can use it to record accounting entries, to generate income statements, to perfrom revenue/expense analysis and to interact with user-defined budget. 

## 2. Funtions

Logged in with different indentity, operator can use indentity-related functions of this system.

### 2.1 Administrator (mode 1)
Operator can enjoy full features of this system.
1. regist or remove multiple new user accounts
2. log in new entries to General Ledger/delete certain entries from General Ledger
3. generate the income statement/compare the financial status for specified months and receive coresponding advice
4. set or reset budget for specified month
5. compare the actual revenue/expense to budget for specified month and receive coresponding advice

### 2.2 User (mode 2)
Operator can enjoy limited features of this system.
1. log in new entries to General Ledger
2. generate the income statement
3. compare the actual revenue/expense to budget for specified month and receive coresponding advice

* User can not access to user account management module. (system security)
* User can not delete certain entries from General Ledger. (internal control requirement)
* User can not modify budget. (internal control requirement)
* User can not compare the financial status for specified months. (limit information access)

* Administrator can also log in as user 0. (In this case, features will be limited.)

## 3. User Information Initialization (mode 0)
After being verified with producer-pre-defined master key. Operator can reset the administrator password and delete all users' account.
* General Ledger and Budget information will not be affected.

## 4. Suggestion for Software Testing
First of all, please put the folders "user_info" "accounting_entries" "budget" (and their temp folders) in the same folder of this program. It contains some pre-enter data for you to test without providing extra informations.

1. The master key is 8Y5I4L2M for this edition. (Pre-defined by producer)
2. Operator can use administrator password 123456 and user password 111111 as user 101.  
(Operator can also regist more or initialize the account.)
3. "Accounting entries" consists 21 entries from 01/2022 to 03/2022.
4. "Description" for entries now consists operating revenue, operating cost, operating expense, non operating revenue and non operating expense. These accounting accounts are not pre-defined and operator can use any other accounting accounts whatever they want. (These categories will be grouped and reported in income statement)
5. Budget is defined for 01/2022, 02/2022, 04/2022.

In [12]:
## import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as ps
import os
import shutil
from datetime import date

session = SparkSession.builder.appName('automatic accountant assistant').getOrCreate()

### base-module 1: reading 
### a solution for the problem being unable to write a csv using pyspark ".coalesce().write" on the same csv file reading before 
### The code is originate by me, but the idea inspired by https://stackoverflow.com/questions/62988178/usage-of-spark-catalog-refreshtabletablename-in-s3
def create_temp(folder_name): # create a new temp folder
    if not os.path.exists(folder_name):
        print("Good, the",folder_name, "folder does not exist. Create a temp folder.")
        os.mkdir(folder_name)
    else:
        print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents. Then, create a temp folder.")
        shutil.rmtree(folder_name) 
        os.mkdir(folder_name)

def copy_allfiles(source, destination): # moves csv inside with the unknown name to temp folder.
    source_files = os.listdir(source)
    for file_name in source_files:
        full_file_name = os.path.join(source, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, destination)
    print("Temp folder exists, proceeding to copy the content. \nReading in csv files...")
### After that we can read in the csv located in temp folder, in order to avoid reading the original csv.
###

### base-module 2: checking whether the user's intended password is valid.
def check_pincode_input():
    while True:
            pincode = input("Please enter new password: (6-digits number):")
            if pincode.isdigit(): #all digits
                if len(pincode) < 6: 
                    print("Too few digits. Key in exactly 6 digits." + "\n")
                elif len(pincode) > 6:
                    print("Too many digits. Key in exactly 6 digits." + "\n")
                elif len(pincode) == 6: 
                    break
            else: # not digits
                print("Only digits are allowed." + "\n")
    return pincode  
###

### main modules (1-4)

### Module 1: User Management system 
### Functions: 1. admin_initialization() 2. registration(1) 3.deletion(1) 4.vertification("admin")/vertification("user")
def admin_initialization(): # reset password of administrator and delete all other users' information
    column1 = []
    column2 = []
    column3 = []
    headers = ['user_ID', 'password', 'user_type']
    
    column1.append("0")
    
    print("Reset administrator password...")
    password_input = check_pincode_input()
    column2.append(password_input)
    
    column3.append("admin")
    
    current_df = session.createDataFrame(zip(column1, column2, column3), headers)
    current_df.show()
    
    #confirmation
    while True:
        conf = input("All existing users will be removed. Do you want to save the change? (Y/N)")
        if conf == "Y":
            break
        elif conf == "N":
            print("The change is not saved. Return to working mode selection menu.\n")
            return
    
    #I need to define save_output() within fuctions everytime I use, otherwise, it generates error.
    def save_output(folder_name,file_type):
        if not os.path.exists(folder_name): # if folder does not exist
            print("Good, the",folder_name, "folder does not exist.")
            current_df.coalesce(1)\
            .write.format(file_type)\
            .option("header","True")\
            .save(folder_name) # save csv file into that particular folder
            print("Successfully saved",file_type,"file into",folder_name, "folder.")
        else:
            print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
            shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
            print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
            save_output(folder_name, file_type) # checking again by calling itself
            
    save_output("user_info","csv")
    print("User information has been initialized. Return to working mode selection menu.\n")
    
    
def registration(number_of_new_user): # regist multiple new user accounts
    number = 0 
    column1 = []
    column2 = []
    column3 = []
    headers = ['user_ID', 'password', 'user_type']
    
    while number < number_of_new_user:
        id_input = input("Please enter your user ID:")
        column1.append(id_input)

        password_input = check_pincode_input()
        column2.append(password_input)

        column3.append("user")
             
        number += 1
    
    reg_df = session.createDataFrame(zip(column1, column2, column3), headers)
    reg_df.show()
    
    ### apply the module "reading"
    create_temp("user_info_temp")              
    copy_allfiles("user_info","user_info_temp")
    
    read_df = session.read.csv('user_info_temp/*.csv',inferSchema=True,header=True)
    ###

    current_df = read_df.union(reg_df) # attach the new users' info. with the existing users' info.
    current_df.show(current_df.count()) # show all users' info. for the operater to check
    
    #confirmation
    while True:
        conf = input("The existing users after operation are shown as above. Do you want to save the change? (Y/N)")
        if conf == "Y":
            break
        elif conf == "N":
            print("The change is not saved. Return to function selection menu.\n")
            return

    def save_output(folder_name,file_type):
        if not os.path.exists(folder_name): # if folder does not exist
            print("Good, the",folder_name, "folder does not exist.")
            current_df.coalesce(1)\
            .write.format(file_type)\
            .option("header","True")\
            .save(folder_name) # save csv file into that particular folder
            print("Successfully saved",file_type,"file into",folder_name, "folder.")
        else:
            print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
            shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
            print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
            save_output(folder_name, file_type) # checking again by calling itself
    save_output("user_info","csv")
    print("The change is saved. Return to function selection menu.\n")


def deletion(number_of_user_to_be_removed): # remove multiple user accounts
    
    ### apply the module "reading"        
    create_temp("user_info_temp")                
    copy_allfiles("user_info","user_info_temp")
    
    read_df = session.read.csv('user_info_temp/*.csv',inferSchema=True,header=True)
    read_df.show(read_df.count()) # show all users' info. for the operater to choose
    ###
    
    read_df_ps = read_df.toPandas() # use Panda on spark to locate certain rows
    
    number = 0 
    while number < number_of_user_to_be_removed:
        del_ID = input("The existing users are shown as above. Please enter the user_ID which you want to delete:")
        while True:
            if del_ID == "0":
                print("You cannot delete the administrator account.")
                del_ID = input("The existing users are shown as above. Please enter the user_ID which you want to delete:")
            else:
                break
        read_df_ps.drop(index=(read_df_ps.loc[read_df_ps['user_ID'] == int(del_ID) ].index), inplace = True) # remove specified user account
        print("Your input has been recorded.")
        number += 1
    
    current_df = session.createDataFrame(read_df_ps)
    current_df.show(current_df.count())
    
    #confirmation
    while True:
        conf = input("The existing users after operation are shown as above. Do you want to save the change? (Y/N)")
        if conf == "Y":
            break
        elif conf == "N":
            print("The change is not saved. Return to function selection menu.\n")
            return
        
    def save_output(folder_name,file_type):
        if not os.path.exists(folder_name): # if folder does not exist
            print("Good, the",folder_name, "folder does not exist.")
            current_df.coalesce(1)\
            .write.format(file_type)\
            .option("header","True")\
            .save(folder_name) # save csv file into that particular folder
            print("Successfully saved",file_type,"file into",folder_name, "folder.")
        else:
            print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
            shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
            print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
            save_output(folder_name, file_type) # checking again by calling itself
    save_output("user_info","csv")
    print("The change is saved. Return to function selection menu.\n")

def vertification(user_type): # verify user(or admin)'s password when operator attempts to log in 
    read_df = session.read.csv('user_info/*.csv',inferSchema=True,header=True)
    read_df_ps = read_df.toPandas() # use Panda on spark to locate certain rows
    
    if user_type == "admin":
        check_ID = "0" # admin's ID is pre-defined to be 0 as above
        password_input = input("Please enter the administrator password:")
    else:
        check_ID = input("Please enter the user_ID which you want to log in:")
        password_input = input("Please enter your user password:") 
    try:
        if int(password_input) == read_df_ps["password"][read_df_ps["user_ID"] == int(check_ID)].values[0]: # verify user's password according to his ID (if that ID exists)
            return True
        else:
            print("Wrong Password.")
            return False
    except IndexError: # IndexError stems from the fact that no such user ID in dataset
        print("User does not exist.")
        return False

### Module 2: General Ledger system
### Functions: 1. new_entry(1) 2.deletion_entries(1)
def new_entry(number_of_new_entry): # log in new entries to General Ledger
    number = 0 
    column1 = []
    column2 = []
    column3 = []
    column4 = []
    column5 = []
    column6 = []
    column7 = []
    headers = ['serial_number', 'year', 'month', 'day', "revenue/expense", "description", "amount"]
    
    while number < number_of_new_entry:
        print("\nYou are typing in the {} entry.".format(number+1))
                
        serial_num = input("Please enter the serial number for this transaction:") # a user-defined unique serial number for each transaction
        column1.append(int(serial_num))
        
        temp_date_input = input("Please enter transaction date (in DD/MM/YYYY format):")
        temp_date = date(int(temp_date_input[6:10]), int(temp_date_input[3:5]), int(temp_date_input[0:2])) # extract info. from input
        print("You have entered: ", "YEAR", temp_date.year, "MONTH", temp_date.month, "DAY", temp_date.day )
        column2.append(temp_date.year)
        column3.append(temp_date.month)
        column4.append(temp_date.day)
        
        desc_input = input("Please enter description for this entry:") # corresponding account entry's name (e.g. operating expense)
        column6.append(desc_input)
        
        while True:
            re_input = input("Is it a revenue entry or an expense entry?(R/E)")
        
            if re_input == "R":
                column5.append("revenue")
                amount = input ("Please enter the amount of this transaction:")
                column7.append(float(amount))
                break                
            elif re_input == "E":
                column5.append("expense")
                amount = input ("Please enter the amount of this transaction:")
                column7.append(-(float(amount)))
                break
                
        print("Your input has been recorded.")     
        number += 1
    
    new_entry_df = session.createDataFrame(zip(column1, column2, column3, column4, column5, column6, column7), headers)
    new_entry_df.show()
    
    # sence 1: There is existing accounting records. Attach the new entries with the existing entries."
    try:
        ### apply the module "reading"         
        create_temp("accounting_entries_temp")            
        copy_allfiles("accounting_entries","accounting_entries_temp")
        
        read_df = session.read.csv('accounting_entries_temp/*.csv',inferSchema=True,header=True)
        ###
        
        current_df = read_df.union(new_entry_df) # Attach the new entries with the existing entries.
        current_df.show(current_df.count())
            
        #confirmation
        while True:
            conf = input("The accounting records after addition are shown as above. Do you want to save the change? (Y/N)")
            if conf == "Y":
                break
            elif conf == "N":
                print("The change is not saved. Return to function selection menu.\n")
                return

        def save_output(folder_name,file_type):
            if not os.path.exists(folder_name): # if folder does not exist
                print("Good, the",folder_name, "folder does not exist.")
                current_df.coalesce(1)\
                .write.format(file_type)\
                .option("header","True")\
                .save(folder_name) # save csv file into that particular folder
                print("Successfully saved",file_type,"file into",folder_name, "folder.")
            else:
                print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
                shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
                print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
                save_output(folder_name, file_type) # checking again by calling itself
        save_output("accounting_entries","csv")
        print("The new entries are saved. Return to function selection menu.\n")
        
        # sence 2: It is the first time the enterprise runs this system. There is no existing accounting records. Just save these new entries."
    except FileNotFoundError:
        print("No existing accounting entries. Create a new file.")
        def save_output(folder_name,file_type):
            if not os.path.exists(folder_name): # if folder does not exist
                print("Good, the",folder_name, "folder does not exist.")
                new_entry_df.coalesce(1)\
                .write.format(file_type)\
                .option("header","True")\
                .save(folder_name) # save csv file into that particular folder
                print("Successfully saved",file_type,"file into",folder_name, "folder.")
            else:
                print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
                shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
                print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
                save_output(folder_name, file_type) # checking again by calling itself
        save_output("accounting_entries","csv")
        print("New accounting entries are generated. Return to function selection menu.\n")

def deletion_entries(number_of_entries_to_be_removed): # delete certain entries from General Ledger
    
    ### apply the module "reading"          
    create_temp("accounting_entries_temp")             
    copy_allfiles("accounting_entries","accounting_entries_temp")
    
    read_df = session.read.csv('accounting_entries_temp/*.csv',inferSchema=True,header=True)
    read_df.show(read_df.count())
    ###
    
    read_df_ps = read_df.toPandas() # use Panda on spark to locate certain rows
    
    number = 0 
    while number < number_of_entries_to_be_removed:
        del_SE = input("The existing entries are shown as above. Please enter the serial_number of the entry which you want to delete:")
        read_df_ps.drop(index=(read_df_ps.loc[read_df_ps['serial_number'] == int(del_SE) ].index), inplace = True) # remove certain row with specified unique serial number
        print("Your input has been recorded.")
        number += 1
    
    current_df = session.createDataFrame(read_df_ps)
    current_df.show(current_df.count())
    
    #confirmation
    while True:
        conf = input("The existing entries after operation are shown as above. Do you want to save the change? (Y/N)")
        if conf == "Y":
            break
        elif conf == "N":
            print("The change is not saved. Return to function selection menu.\n")
            return
        
    def save_output(folder_name,file_type):
        if not os.path.exists(folder_name): # if folder does not exist
            print("Good, the",folder_name, "folder does not exist.")
            current_df.coalesce(1)\
            .write.format(file_type)\
            .option("header","True")\
            .save(folder_name) # save csv file into that particular folder
            print("Successfully saved",file_type,"file into",folder_name, "folder.")
        else:
            print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
            shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
            print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
            save_output(folder_name, file_type) # checking again by calling itself
    save_output("accounting_entries","csv")
    print("The change is saved. Return to function selection menu.\n")

### Module 3: Financial Statement (and Statement Analysis) system
### Functions: 1. get_monthly_report() 2.monthly_comparison()
def get_monthly_report(): # generate the income statement
    temp_date_input = input("Please enter report period (in MM/YYYY format):")
    year = str(int(temp_date_input[3:7])) # extract info. from input
    month = str(int(temp_date_input[0:2]))
    print("You have entered: ", "YEAR", year, "MONTH", month)
    read_df = session.read.csv('accounting_entries_temp/*.csv',inferSchema=True,header=True)
    monthly_report_df = read_df.filter((read_df["year"] == year) & (read_df["month"] == month)) # filter entries in specified months
    
    print("In this month, our company has {} transactions as shown below.".format(monthly_report_df.count()))
    
    monthly_report_df.show(monthly_report_df.count())
    
    monthly_report_description = monthly_report_df.groupBy('description').agg({'amount':'sum'}).collect() # aggregate amounts groupby account entry's name
    monthly_report_RE = monthly_report_df.groupBy('revenue/expense').agg({'amount':'sum'}).collect() # aggregate amounts groupby revenue/expense
    monthly_report_NI = monthly_report_df.agg({'amount':'sum'}).collect() # aggregate amounts for the whole month
    
    print("Here is the income statement for {}".format(temp_date_input) + "\n")
    for row in monthly_report_description:
        print(str(row["description"]) + ": " + str(row["sum(amount)"])) # report the amount with account entry's name
    print()
    for row in monthly_report_RE: 
        print(str(row["revenue/expense"]) + ": " + str(row["sum(amount)"])) # report the amount with revenue/expense
    print()
    print("Net income: " + str([row["sum(amount)"] for row in monthly_report_NI][0])) # report net income
    print("\nIncome Statement Generated. Return to function selection menu.\n")

def monthly_comparison(): # compare the financial status for specified months
    temp_date_input_2 = input("Please enter the analysis period (in MM/YYYY format):")
    year2 = str(int(temp_date_input_2[3:7]))
    month2 = str(int(temp_date_input_2[0:2]))
    print("The analysis period is: ", "YEAR", year2, "MONTH", month2)
    read_df_2 = session.read.csv('accounting_entries/*.csv',inferSchema=True,header=True)
    monthly_report_df_2 = read_df_2.filter((read_df_2["year"] == year2) & (read_df_2["month"] == month2)) # filter entries in specified months
    monthly_report_NI_2 = monthly_report_df_2.agg({'amount':'sum'}).collect() # aggregate amounts for the whole month
    net_income_2 = [row["sum(amount)"] for row in monthly_report_NI_2][0] # get net income of analysis month
    
    temp_date_input_1 = input("Please enter the base period (in MM/YYYY format):")
    year1 = str(int(temp_date_input_1[3:7])) 
    month1 = str(int(temp_date_input_1[0:2]))
    print("The base period is: ", "YEAR", year1, "MONTH", month1)
    read_df_1 = session.read.csv('accounting_entries/*.csv',inferSchema=True,header=True)
    monthly_report_df_1 = read_df_1.filter((read_df_1["year"] == year1) & (read_df_1["month"] == month1)) # filter entries in specified months
    monthly_report_NI_1 = monthly_report_df_1.agg({'amount':'sum'}).collect() # aggregate amounts for the whole month
    net_income_1 = [row["sum(amount)"] for row in monthly_report_NI_1][0] # get net income of base month
    
    print("\nConcerning net income...") # compare the net income for specified months and give advice
    if net_income_1 == net_income_2: # if the net incomes are the same
        print("Comparing with the net income {} in period {} , the net income {} in period {} is the same. Keep going!"\
              .format(net_income_1, temp_date_input_1, net_income_2, temp_date_input_2))
    elif net_income_1 == 0 and net_income_2 > 0: # net income in the analysis month is positive, while net income in the base month is zero
        print("The net income in period {} is zero and the net income in period {} is {}. Good job!"\
              .format(temp_date_input_1, temp_date_input_2, net_income_2))
    elif net_income_1 == 0 and net_income_2 < 0: # net income in the analysis month is negative, while net income in the base month is zero
        print("The net income in period {} is zero and The net income in period {} is {}. Think more!"\
              .format(temp_date_input_1, temp_date_input_2, net_income_2))
    elif net_income_2 > net_income_1 and net_income_1 > 0: # The net income in the analysis month is more, while net income in the base month is above zero
        print("Comparing with the net income {} in period {}, the net income {} in period {} increases by {:.2f} percents. Good job!"\
              .format(net_income_1, temp_date_input_1, net_income_2, temp_date_input_2, (net_income_2/net_income_1 - 1)*100))
    elif net_income_2 > net_income_1 and net_income_1 < 0: # The net income in the analysis month is more, while net income in the base month is below zero
        print("Comparing with the negative net income {} in period {}, we realize positive net income {} in period {}. Good job!"\
              .format(net_income_1, temp_date_input_1, net_income_2, temp_date_input_2))       
    elif net_income_2 < net_income_1: # The net income in the analysis month is less than net income in the base month
        print("Comparing with the net income {} in period {}, the net income {} in period {} decreases by {:.2f} percents. Think more!"\
              .format(net_income_1, temp_date_input_1, net_income_2, temp_date_input_2, -(net_income_2/net_income_1 - 1)*100))
    
    monthly_report_RE_1 = monthly_report_df_1.groupBy('revenue/expense').agg({'amount':'sum'}).collect()
    monthly_report_RE_2 = monthly_report_df_2.groupBy('revenue/expense').agg({'amount':'sum'}).collect()
    
    def get_revenue_expense(monthly_report_RE): # transform the dataframe of aggregation groupby revenue/expense into dictionary, in order to retrieve the value for comparison
        templist1 = []
        templist2 = []
        for row in monthly_report_RE:
            templist1.append(row["revenue/expense"])
            templist2.append(row["sum(amount)"])
        re_month = dict(zip(templist1,templist2))
        return re_month
    
    dict_RE_1 = get_revenue_expense(monthly_report_RE_1)
    dict_RE_2 = get_revenue_expense(monthly_report_RE_2)
    
    print("Concerning revenue and expense...")
    if dict_RE_2["revenue"] > dict_RE_1["revenue"]: # The revenue in the analysis month is more
        print("Comparing with the revenue {} in period {}, the revenue {} in period {} increases by {:.2f} percents. Good job!"\
              .format(dict_RE_1["revenue"], temp_date_input_1, dict_RE_2["revenue"], temp_date_input_2, (dict_RE_2["revenue"]/dict_RE_1["revenue"] - 1)*100))       
    elif dict_RE_2["revenue"] < dict_RE_1["revenue"]: # The revenue in the analysis month is less
        print("Comparing with the revenue {} in period {}, the revenue {} in period {} decreases by {:.2f} percents. Think more!"\
              .format(dict_RE_1["revenue"], temp_date_input_1, dict_RE_2["revenue"], temp_date_input_2, -(dict_RE_2["revenue"]/dict_RE_1["revenue"] - 1)*100))
    else: # The revenue in the analysis month is the same
        print("Comparing with the revenue {} in period {} , the revenue {} in period {} is the same. Keep going!"\
              .format(dict_RE_1["revenue"], temp_date_input_1, dict_RE_2["revenue"], temp_date_input_2))
    
    if -dict_RE_2["expense"] > -dict_RE_1["expense"]: # The expense in the analysis month is more
        print("Comparing with the expense {} in period {}, the expense {} in period {} increases by {:.2f} percents. Think more!"\
              .format(-dict_RE_1["expense"], temp_date_input_1, -dict_RE_2["expense"], temp_date_input_2, (-dict_RE_2["expense"]/-dict_RE_1["expense"] - 1)*100))       
    elif -dict_RE_2["expense"] < -dict_RE_1["expense"]: # The expense in the analysis month is less
        print("Comparing with the expense {} in period {}, the expense {} in period {} decreases by {:.2f} percents. Good job!"\
              .format(-dict_RE_1["expense"], temp_date_input_1, -dict_RE_2["expense"], temp_date_input_2, -(-dict_RE_2["expense"]/-dict_RE_1["expense"] - 1)*100))
    else: # The expense in the analysis month is the same
        print("Comparing with the expense {} in period {} , the expense {} in period {} is the same. Keep going!"\
              .format(-dict_RE_1["expense"], temp_date_input_1, -dict_RE_2["expense"], temp_date_input_2))
    
    print("\nAnalysis Finished. Return to function selection menu.\n")

### Module 4: Budget (and Budget Analysis) system
### Functions: 1. budget_setting() 2. budget_reset() 3. budget_checking()
def budget_setting(): # set budget for specified month
    column1 = []
    column2 = []
    column3 = []
    column4 = []
    column5 = []
    headers = ['year', 'month', "revenue/expense", "amount", "reference_num"]
    
    temp_date_input = input("Please enter the month that you want to set budget for (in MM/YYYY format):")
    year = str(int(temp_date_input[3:7])) 
    month = str(int(temp_date_input[0:2]))
    print("You are setting budget for: ", "YEAR", year, "MONTH", month)
    column1.append(year)
    column2.append(month)
    

    column3.append("revenue")
    amount = input("Please enter the amount of expected revenue for that month:")
    column4.append(float(amount))
    column5.append(str(year) + str(month) + str(0)) # generate a unique reference number for this entry. num = "year" + "month" + "revenue (0)"  
    
    column1.append(year)
    column2.append(month)

    column3.append("expense")
    amount = input("Please enter the amount of expected expense for that month:")
    column4.append((float(amount)))
    column5.append(str(year) + str(month) + str(1)) # generate a unique reference number for this entry. num = "year" + "month" + "expense (1)"
    
    new_budget_df = session.createDataFrame(zip(column1, column2, column3, column4, column5), headers)
    new_budget_df.show()
    
     #sence 1: There is existing budget records. Attach the new budget records with the existing budget records."
    try:
        ### apply the module "reading"                  
        create_temp("budget_temp")                    
        copy_allfiles("budget","budget_temp")
        
        read_df = session.read.csv('budget_temp/*.csv',inferSchema=True,header=True)
        ###
        
        current_df = read_df.union(new_budget_df) # Attach the new budget records with the existing budget records
        current_df.show(current_df.count())
            
        #confirmation
        while True:
            conf = input("The budget after addition are shown as above. Do you want to save the change? (Y/N)")
            if conf == "Y":
                break
            elif conf == "N":
                print("The change is not saved. Return to function selection menu.\n")
                return

        def save_output(folder_name,file_type):
            if not os.path.exists(folder_name): # if folder does not exist
                print("Good, the",folder_name, "folder does not exist.")
                current_df.coalesce(1)\
                .write.format(file_type)\
                .option("header","True")\
                .save(folder_name) # save csv file into that particular folder
                print("Successfully saved",file_type,"file into",folder_name, "folder.")
            else:
                print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
                shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
                print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
                save_output(folder_name, file_type) # checking again by calling itself
        save_output("budget","csv")
        print("The new budget file is saved. Return to function selection menu.\n")
        
        #sence 2: It is the first time the enterprise runs this system. There is no existing budget records. Just save these new budget records." 
    except FileNotFoundError:
        print("No existing budget file. Create a new file.")
        def save_output(folder_name,file_type):
            if not os.path.exists(folder_name): # if folder does not exist
                print("Good, the",folder_name, "folder does not exist.")
                new_budget_df.coalesce(1)\
                .write.format(file_type)\
                .option("header","True")\
                .save(folder_name) # save csv file into that particular folder
                print("Successfully saved",file_type,"file into",folder_name, "folder.")
            else:
                print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
                shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
                print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
                save_output(folder_name, file_type) # checking again by calling itself
        save_output("budget","csv")
        print("The new budget file is generated. Return to function selection menu.\n")

def budget_reset(): # delete the existing budget and set new budget for specified month
    
    ### apply the module "reading" 
    create_temp("budget_temp")            
    copy_allfiles("budget","budget_temp")
    
    read_df = session.read.csv('budget_temp/*.csv',inferSchema=True,header=True)
    read_df.show(read_df.count())
    ###
    
    read_df_ps = read_df.toPandas()
    
    temp_date_input = input("The existing budget info. is shown as above. Please enter the month that you want to reset budget for (in MM/YYYY format):")
    year = str(int(temp_date_input[3:7])) 
    month = str(int(temp_date_input[0:2]))
    
    # remove the budget record according to unique reference number (num = "year" + "month" + "revenue(0)/expense (1)")
    read_df_ps.drop(index=(read_df_ps.loc[read_df_ps['reference_num'] == int(str(year)+str(month)+str(0))].index), inplace = True) 
    read_df_ps.drop(index=(read_df_ps.loc[read_df_ps['reference_num'] == int(str(year)+str(month)+str(1))].index), inplace = True)
    print("Your input has been recorded.")
    
    del_df = session.createDataFrame(read_df_ps)
    
    # set new budget for specified month
    column1 = []
    column2 = []
    column3 = []
    column4 = []
    column5 = []
    headers = ['year', 'month', "revenue/expense", "amount", "reference_num"]
    
    print("You are resetting budget for: ", "YEAR", year, "MONTH", month)
    column1.append(year)
    column2.append(month)
    

    column3.append("revenue")
    amount = input("Please enter the amount of expected revenue for that month:")
    column4.append(float(amount))
    column5.append(str(year) + str(month) + str(0))
    
    column1.append(year)
    column2.append(month)

    column3.append("expense")
    amount = input("Please enter the amount of expected expense for that month:")
    column4.append((float(amount)))
    column5.append(str(year) + str(month) + str(1))
    
    new_budget_df = session.createDataFrame(zip(column1, column2, column3, column4, column5), headers)
    
    current_df = del_df.union(new_budget_df) # Attach the new budget records with the existing budget records after deletion
    current_df.show(current_df.count())
    
    
    #confirmation
    while True:
        conf = input("The budget after addition are shown as above. Do you want to save the change? (Y/N)")
        if conf == "Y":
            break
        elif conf == "N":
            print("The change is not saved. Return to function selection menu.\n")
            return
        
    def save_output(folder_name,file_type):
        if not os.path.exists(folder_name): # if folder does not exist
            print("Good, the",folder_name, "folder does not exist.")
            current_df.coalesce(1)\
            .write.format(file_type)\
            .option("header","True")\
            .save(folder_name) # save csv file into that particular folder
            print("Successfully saved",file_type,"file into",folder_name, "folder.")
        else:
            print("Detected that",folder_name, "folder exists, proceeding to delete folder and its contents.")
            shutil.rmtree(folder_name) # if folder exists, remove folder and all its contents
            print("Since we cannot write into the same folder, we removed",folder_name, "folder and its contents.")
            save_output(folder_name, file_type) # checking again by calling itself
    save_output("budget","csv")
    print("The change is saved. Return to function selection menu.\n")

def budget_checking(): # compare the actual revenue/expense to budget for specified month and give advice
    try:
        temp_date_input = input("Please enter the current budget period (in MM/YYYY format):")
        year = str(int(temp_date_input[3:7])) #extract info. from input
        month = str(int(temp_date_input[0:2]))
        print("The current budget period is: ", "YEAR", year, "MONTH", month)
        read_df_b = session.read.csv('budget/*.csv',inferSchema=True,header=True)
        monthly_report_df_b = read_df_b.filter((read_df_b["year"] == year) & (read_df_b["month"] == month))
        monthly_report_RE_b = monthly_report_df_b.groupBy('revenue/expense').agg({'amount':'sum'}).collect()
    
        read_df_1 = session.read.csv('accounting_entries/*.csv',inferSchema=True,header=True)
        monthly_report_df_1 = read_df_1.filter((read_df_1["year"] == year) & (read_df_1["month"] == month))
        monthly_report_RE_1 = monthly_report_df_1.groupBy('revenue/expense').agg({'amount':'sum'}).collect()
    
        def get_revenue_expense(monthly_report_RE): # transform the dataframe of aggregation groupby revenue/expense into dictionary, in order to retrieve the value for comparison
            templist1 = []
            templist2 = []
            for row in monthly_report_RE:
                templist1.append(row["revenue/expense"])
                templist2.append(row["sum(amount)"])
            re_month = dict(zip(templist1,templist2))
            return re_month
    
        dict_RE_b = get_revenue_expense(monthly_report_RE_b)
        dict_RE_1 = get_revenue_expense(monthly_report_RE_1)
    
        print("Concerning budget for revenue...") # compare the actual revenue to budget for specified month and give advice
        if dict_RE_1["revenue"] > dict_RE_b["revenue"]: # realize more revenue than budget
            print("The current revenue is {}. The company has finished the budget revenue {} and earned more by {:.2f} percents. Good job!"\
                  .format(dict_RE_1["revenue"], dict_RE_b["revenue"], (dict_RE_1["revenue"]/dict_RE_b["revenue"] - 1)*100))       
        elif dict_RE_1["revenue"] < dict_RE_b["revenue"]: # realize less revenue than budget
            print("The current revenue is {}. The company need to collect more {} ({:.2f} percents of budget) to catch up with the target."\
                  .format(dict_RE_1["revenue"], dict_RE_b["revenue"]-dict_RE_1["revenue"], (1 - dict_RE_1["revenue"]/dict_RE_b["revenue"])*100))
        else: # realized revenue equal budget
            print("The current revenue is {}. The company has finished the budget revenue. Keep going!"\
                  .format(dict_RE_1["revenue"]))
    
        print("Concerning budget for expense...") # compare the actual expense to budget for specified month and give advice
        if -dict_RE_1["expense"] > dict_RE_b["expense"]: # spend more expense than budget
            print("The current expense is {}. The company ran out the budget expense {} and spent more by {:.2f} percents. Please be careful of and try to refrain from further expense."\
                  .format(dict_RE_1["expense"], dict_RE_b["expense"], (-dict_RE_1["expense"]/dict_RE_b["expense"] - 1)*100))       
        elif -dict_RE_1["expense"] == dict_RE_b["expense"]: # spend expense as much as budget  
            print("The current expense is {}. The company has run out the budget. Please be careful of and try to refrain from further expense."\
                  .format(dict_RE_1["expense"]))
        elif -dict_RE_1["expense"] > 0.5 * dict_RE_b["expense"]: # little budget remaining # P.S. dict_RE_1["expense"] is negative while dict_RE_b["expense"] is positive by my setting before
            print("The current expense is {}. The company has only {} ({:.2f} percents of budget) to spend within the budget. Please limit unnessary expense."\
                  .format(dict_RE_1["expense"], dict_RE_b["expense"]+dict_RE_1["expense"], (1 + dict_RE_1["expense"]/dict_RE_b["expense"])*100))
        elif -dict_RE_1["expense"] > 0: # much budget remaining
            print("The current expense is {}. The company still has {} ({:.2f} percents of budget) to spend within the budget."\
                  .format(dict_RE_1["expense"], dict_RE_b["expense"]+dict_RE_1["expense"], (1 + dict_RE_1["expense"]/dict_RE_b["expense"])*100))
        else: # no expense
            print("The current expense is zero.")
        
        print("The current budget situation and suggestion are shown above. Return to function selection menu.\n")
    except KeyError: # KeyError stems from inexistence of budget for certain month
        print("No applicable data. Please select a month in which budget have already been set and entries have already been recorded. Return to function selection menu.\n")

################################################################################################################################################################################
### main program
master_key_def = "8Y5I4L2M" # pre-defined for identity verification in user information initialization

print("Welcome to automatic accountant assistant (Version 1.0).")

while True:
    mode = input("""Please enter the working mode code:
    1 for logging in as administrator;
    2 for logging in as user;
    
    0 for user information initialization (need to verify master key);
    If you want to exit, press the Enter key without typing anything.
    """)
    
    if mode == "0": 
        print("initializtion")
        master_key = input("Please enter the master key:")
        if master_key == master_key_def:
            admin_initialization()
        else:
            print("Master key does not match. Return to mode selection menu.") 
            
    elif mode == "1":
        print("administrator mode")
        if vertification("admin"):
            print("Success! You are currently logged in as administrator.")
            while True:
                mode_admin = input("""Please enter the code to select function: 
                11 for new user registration; 12 for existing user deletion;
                21 for logging in new accounting entries; 22 for existing entries deletion;
                31 for generating financial statements; 32 for comparison and advice on financial status between months;
                41 for budget setting; 42 for budget resetting; 43 for current budget status checking (with advice)
                
                If you want to return to working mode selection menu, press the Enter key without typing anything.
                """)
                
                if mode_admin == "11":
                    num_reg = input("Please enter the number of new users:")
                    registration(int(num_reg))
                elif mode_admin == "12":
                    num_del = input("Please enter the number of users which you want to delete:")
                    deletion(int(num_del))
                elif mode_admin == "21":
                    num_entries = input("Please enter the number of new entries:")
                    new_entry(int(num_entries))
                elif mode_admin == "22":
                    number_del_ent = input("Please enter the number of entries which you want to delete:")     
                    deletion_entries(int(number_del_ent))
                elif mode_admin == "31":   
                    get_monthly_report()
                elif mode_admin == "32":   
                    monthly_comparison()
                elif mode_admin == "41":
                    budget_setting()
                elif mode_admin == "42":
                    budget_reset()
                elif mode_admin == "43":
                    budget_checking()                
                    
                elif mode_admin == "":
                    print("Return to working mode selection menu.\n")
                    break
                else:
                    print("Please enter a valid mode code.")
        else:
            print("Return to working mode selection menu.\n")
            
    elif mode == "2":
        print("user mode")
        if vertification("user"):
            print("Success! You are currently logged in as user.")
            while True:
                mode_user = input("""Please enter the code to select function: 
                1 for logging in new accounting entries; 
                2 for generating financial statements;
                3 for current budget status checking (with advice)
                
                If you want to return to working mode selection menu, press the Enter key without typing anything.
                """)
                
                if mode_user == "1":
                    num_entries = input("Please enter the number of new entries:")
                    new_entry(int(num_entries))
                elif mode_user == "2":
                    get_monthly_report()
                elif mode_user == "3":
                    budget_checking()
                elif mode_user == "":
                    print("Return to working mode selection menu.\n")
                    break
                else:
                    print("Please enter a valid mode code.")
        else:
            print("Return to working mode selection menu.\n")
            
    elif mode == "":
        print("Thank you for using automatic accountant assistant (Version 1.0). See you!")
        break
    else:
        print("Please enter a valid mode code.")

Welcome to automatic accountant assistant (Version 1.0).


Please enter the working mode code:
    1 for logging in as administrator;
    2 for logging in as user;
    
    0 for user information initialization (need to verify master key);
    If you want to exit, press the Enter key without typing anything.
     1


administrator mode


Please enter the administrator password: 123456


Success! You are currently logged in as administrator.


...

The current budget period is:  YEAR 2022 MONTH 2
Concerning budget for revenue...
The current revenue is 1300.0. The company need to collect more 800.0 (38.10 percents of budget) to catch up with the target.
Concerning budget for expense...
The current expense is -1440.0. The company has only 660.0 (31.43 percents of budget) to spend within the budget. Please limit unnessary expense.
The current budget situation and suggestion are shown above. Return to function selection menu.



Please enter the code to select function: 
                1 for logging in new accounting entries; 
                2 for generating financial statements;
                3 for current budget status checking (with advice)
                
                If you want to return to working mode selection menu, press the Enter key without typing anything.
                 


Return to working mode selection menu.



Please enter the working mode code:
    1 for logging in as administrator;
    2 for logging in as user;
    
    0 for user information initialization (need to verify master key);
    If you want to exit, press the Enter key without typing anything.
     


Thank you for using automatic accountant assistant (Version 1.0). See you!
